# Yellow pages 

## Import libraries

In [1]:
import requests
from lxml import html
import unicodecsv as csv
import argparse
import datetime
import pandas as pd
from collections import OrderedDict, deque
import re

## Define parameters function

- Function to set parameters for search on yellow pages. Set parameter to True to include in search. Must haveat least one true for each parameter

In [2]:

def parameters():
	keyword_dict = {
		'Dentists': True
	}

	city_province_dict = {
		'toronto+ON': True
	}

	pagination_list=[*range(1,5)]
	
	if sum(keyword_dict.values()) == 0 or sum(city_province_dict.values()) == 0:
		sys.exit("Ensure you have specified at least one keyword and city, province")
	else:
		return keyword_dict, city_province_dict, pagination_list

## Define crawl function

Function to process yellowpage listing page 
- param keyword: search query
- param place_city : place city name
- param place_province: place province name

In [3]:
pagination='3'
keyword='Dentists'
place_city_province='toronto+ON'
# url = "https://www.yellowpages.ca/search/si/3/{0}/{1}".format(keyword, place_city_province )
url = "https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON"
	# /pagenumber/keyword/placecity+placeprovince

print("retrieving ",url)

headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
			'Accept-Encoding':'gzip, deflate, br',
			'Accept-Language':'en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7',
			'Cache-Control':'max-age=0',
			'Connection':'keep-alive',
			'Host':'www.yellowpages.ca',
			'Upgrade-Insecure-Requests':'1',
			'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
		}

# Adding retries
for retry in range(2):
	try:
		response = requests.get(url,verify=False, headers = headers )

		if response.status_code==200:
			parser = html.fromstring(response.text)

			#making links absolute
			base_url = "https://www.yellowpages.ca"
			parser.make_links_absolute(base_url)

			XPATH_LISTINGS = "//div[contains(@class, 'listing listing--bottomcta placement')]" #listings
			listings = parser.xpath(XPATH_LISTINGS)
			scraped_results = []

			for results in listings:

				XPATH_BUSINESS_NAME = ".//a[contains(@class, 'listing__name--link')]//text()"
				# XPATH_BUSINESS_PAGE = ".//a[contains(@class, 'listing__name--link')]//@href"
				XPATH_STREET = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='streetAddress']//text()"
				XPATH_LOCALITY = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='addressLocality']//text()"
				XPATH_REGION = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='addressRegion']//text()"
				XPATH_ZIP_CODE = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='postalCode']//text()"
				XPATH_WEBSITE = ".//div[contains(@class, 'listing__mlr__root')]//ul//li[3]//a//@href"
				
				XPATH_PHONE= ".//div[contains(@class, 'listing__mlr__root')]//ul//li[1]//a//@data-phone"
				XPATH_DESCRIPTION = ".//article[contains(@class, 'listing__details__teaser')]//text()"
				XPATH_PRODUCTS = ".//div[contains(@class, 'listing__captext')]//text()"


				raw_business_name = results.xpath(XPATH_BUSINESS_NAME)
				# raw_business_page = results.xpath(XPATH_BUSINESS_PAGE)
				raw_website = results.xpath(XPATH_WEBSITE)
				raw_street = results.xpath(XPATH_STREET)
				raw_locality = results.xpath(XPATH_LOCALITY)
				raw_region = results.xpath(XPATH_REGION)
				raw_zip_code = results.xpath(XPATH_ZIP_CODE)

				raw_phone=results.xpath(XPATH_PHONE)
				raw_products=results.xpath(XPATH_PRODUCTS)
				raw_description=results.xpath(XPATH_DESCRIPTION)
				
				business_name = ''.join(raw_business_name).strip() if raw_business_name else None
				# business_page = ''.join(raw_business_page).strip() if raw_business_page else None
				street = ''.join(raw_street).strip() if raw_street else None
				website = ''.join(raw_website).strip() if raw_website else None
				locality = ''.join(raw_locality).replace(',\xa0','').strip() if raw_locality else None
				region = ''.join(raw_region).strip() if raw_region else None
				zipcode = ''.join(raw_zip_code).strip() if raw_zip_code else None

				phone= ''.join(raw_phone).strip() if raw_phone else None
				product=''.join(raw_products).strip() if raw_products else None
				description=''.join(raw_description).strip() if raw_description else None

				business_details = {
									'business_name':business_name,
									# 'business_page':business_page,
									'street': street,
									'website':website,
									'street':street,
									'city':locality,
									'region':region,
									'zipcode':zipcode,
									'industry': keyword,
									'phone': phone,
									'product':product,
									'description':description,
									# 'updated_at': datetime.datetime.now()
				}
				scraped_results.append(business_details)

		

		elif response.status_code==404:
			print("Could not find a location matching")
			#no need to retry for non existing page
			break
		else:
			print("Failed to process page")
		
	except:
		print("Failed to process page")


retrieving  https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON
Failed to process page
Failed to process page


In [4]:
scraped_results

NameError: name 'scraped_results' is not defined

In [5]:

def parse_listing(keyword, place_city_province, pagination=3):
    	
	url = "https://www.yellowpages.ca/search/si/{0}/{1}/{2}".format(pagination,keyword, place_city_province )
	# url = "https://www.yellowpages.ca/search/si/{pagination}/{keyword}/{place_city_province}"
	# /pagenumber/keyword/placecity+placeprovince

	print("retrieving ",url)

	headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
				'Accept-Encoding':'gzip, deflate, br',
				'Accept-Language':'en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7',
				'Cache-Control':'max-age=0',
				'Connection':'keep-alive',
				'Host':'www.yellowpages.ca',
				'Upgrade-Insecure-Requests':'1',
				'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
			}

	# Adding retries
	for retry in range(10):
		try:
			response = requests.get(url,verify=False, headers = headers )

			if response.status_code==200:
				parser = html.fromstring(response.text)

				#making links absolute
				base_url = "https://www.yellowpages.ca"
				parser.make_links_absolute(base_url)

				XPATH_LISTINGS = "//div[contains(@class, 'listing listing--bottomcta placement')]" #listings
				listings = parser.xpath(XPATH_LISTINGS)
				scraped_results = []

				for results in listings:

					XPATH_BUSINESS_NAME = ".//a[contains(@class, 'listing__name--link')]//text()"
					# XPATH_BUSINESS_PAGE = ".//a[contains(@class, 'listing__name--link')]//@href"
					XPATH_STREET = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='streetAddress']//text()"
					XPATH_LOCALITY = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='addressLocality']//text()"
					XPATH_REGION = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='addressRegion']//text()"
					XPATH_ZIP_CODE = ".//div[contains(@class, 'listing__address')]//span[@itemprop='address']//span[@itemprop='postalCode']//text()"
					XPATH_WEBSITE = ".//div[contains(@class, 'listing__mlr__root')]//ul//li[3]//a//@href"
					
					XPATH_PHONE= ".//div[contains(@class, 'listing__mlr__root')]//ul//li[1]//a//@data-phone"
					XPATH_DESCRIPTION = ".//article[contains(@class, 'listing__details__teaser')]//text()"
					XPATH_PRODUCTS = ".//div[contains(@class, 'listing__captext')]//text()"


					raw_business_name = results.xpath(XPATH_BUSINESS_NAME)
					# raw_business_page = results.xpath(XPATH_BUSINESS_PAGE)
					raw_website = results.xpath(XPATH_WEBSITE)
					raw_street = results.xpath(XPATH_STREET)
					raw_locality = results.xpath(XPATH_LOCALITY)
					raw_region = results.xpath(XPATH_REGION)
					raw_zip_code = results.xpath(XPATH_ZIP_CODE)

					raw_phone=results.xpath(XPATH_PHONE)
					raw_products=results.xpath(XPATH_PRODUCTS)
					raw_description=results.xpath(XPATH_DESCRIPTION)
					
					business_name = ''.join(raw_business_name).strip() if raw_business_name else None
					# business_page = ''.join(raw_business_page).strip() if raw_business_page else None
					street = ''.join(raw_street).strip() if raw_street else None
					website = ''.join(raw_website).strip() if raw_website else None
					locality = ''.join(raw_locality).replace(',\xa0','').strip() if raw_locality else None
					region = ''.join(raw_region).strip() if raw_region else None
					zipcode = ''.join(raw_zip_code).strip() if raw_zip_code else None

					phone= ''.join(raw_phone).strip() if raw_phone else None
					product=''.join(raw_products).strip() if raw_products else None
					description=''.join(raw_description).strip() if raw_description else None

					business_details = {
										'business_name':business_name,
										# 'business_page':business_page,
										'street': street,
										'website':website,
										'street':street,
										'city':locality,
										'region':region,
										'zipcode':zipcode,
										'industry': keyword,
										'phone': phone,
										'product':product,
										'description':description,
										# 'updated_at': datetime.datetime.now()
					}
					scraped_results.append(business_details)

				return scraped_results

			elif response.status_code==404:
				print("Could not find a location matching")
				#no need to retry for non existing page
				break
			else:
				print("Failed to process page")
				return []
				
		except:
			print("Failed to process page")
			return []

In [6]:
pagination_list=[*range(1,5)]
for pagination in range(1,len(pagination_list)):
    print(pagination)

1
2
3


In [7]:
pagination_list=[*range(1,5)]
for pagination in range(1,len(pagination_list)):
    print(pagination)
    scraped_data1=parse_listing('Dentists','toronto+ON',pagination)
   

1
retrieving  https://www.yellowpages.ca/search/si/1/Dentists/toronto+ON


/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


2
retrieving  https://www.yellowpages.ca/search/si/2/Dentists/toronto+ON


/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


3
retrieving  https://www.yellowpages.ca/search/si/3/Dentists/toronto+ON


/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
scraped_data1=parse_listing('Dentists','toronto+ON',1)

retrieving  https://www.yellowpages.ca/search/si/1/Dentists/toronto+ON


/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
scraped_data2=parse_listing('Dentists','toronto+ON',2)

retrieving  https://www.yellowpages.ca/search/si/2/Dentists/toronto+ON


/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/ada/anaconda3/envs/uvo/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.yellowpages.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [10]:
pagination_list=[*range(1,5)]
pd.DataFrame(parse_listing('Dentists','toronto+ON',pagination_list=pagination_list)).shape

TypeError: parse_listing() got an unexpected keyword argument 'pagination_list'

In [ ]:
data.shape

In [ ]:
pd.DataFrame(scraped_data2).head()

In [ ]:
pagination_list=[*range(1,5)]
data=[]
for i in range(len(pagination_list)):
    scraped_data=parse_listing('Dentists','toronto+ON',pagination_list[i])
    data.append(scraped_data)
data

In [ ]:
data[2]

## Crawl pages

In [ ]:
keyword_dict, place_city_province_dict,pagination_list= parameters()

keyword_list = [key for key, value in keyword_dict.items() if value]
place_city_province_list = [key for key, value in place_city_province_dict.items() if value]
# pagination_list=[key for key, value in pagination_dict.items() if value]
pagination_l=pagination_list

data=[]
for pagination in pagination_l:
    for place_city_province in place_city_province_list:
        for keyword in keyword_list:
            
            # data=[]
            # for i in range(1,len(pagination_list)):
            scraped_data =  parse_listing(keyword,place_city_province,pagination)
            data.append(scraped_data)
            # df = pd.DataFrame(data[i])

            # df_=pd.concat([df, df])
                
    

## Create dataframe

In [ ]:
def create_dataframe(list):
    flattened  = [val for sublist in data for val in sublist]
    df=pd.DataFrame(flattened)
    sites=df['website'].tolist()
    sites=df['website'].tolist()
    new = []
    for i in sites:
        try:
            val = i.split("%3A%2F%2F")[-1].split("%")[0]
            new.append(val)
        except Exception:
            val = "null"
            new.append(val)
    df['website']=new
    return df
    
df=create_dataframe(data)


In [ ]:
dfs = []
for i in data:
    dfs.append(pd.DataFrame(i))

mega = pd.concat(dfs)
mega.shape

In [ ]:
print(mega.duplicated().sum())

In [ ]:
data[0]

In [ ]:
df['website'].to_csv('email.csv')

## Get emails

In [ ]:
def get_email(url):
    
    unprocessed_urls = deque([url])

    processed_urls = set()

    emails = set()
    url = url

    print("Crawling URL %s" % url)
    response = requests.get(url)

    new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
    new_emails = [mail for mail in new_emails if ".png" not in mail]
    new_emails = [mail for mail in new_emails if ".jpg" not in mail]
    
    emails.update(new_emails)
    
    return emails

In [ ]:
mails = []

for site in df.website.values:
    try:
        mail = get_email("http://" + site)
        
    except Exception:
        try:
            mail = get_email("https://" + site)
        
        except Exception:
            mail = "null"
            pass
    
    mails.append(mail)

In [ ]:
df.shape